# Library

In [1]:
import pandas as pd
import numpy as np
import os
import re
import collections
import unidecode
#from gensim.parsing.preprocessing import remove_stopwords
import nltk
from nltk.corpus import stopwords
import itertools 
from nltk.tokenize import word_tokenize
from string import punctuation

from nltk import ngrams

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [2]:
import numpy as np

import tensorflow_datasets as tfds
import tensorflow as tf

from tensorflow.keras import preprocessing
from tensorflow.keras import layers

tfds.disable_progress_bar()

In [3]:
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
from tensorflow.keras import utils
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

import tensorflow_datasets as tfds
import tensorflow_text as tf_text

In [4]:
pd.set_option('display.max_colwidth', 100)

In [5]:
%matplotlib inline
from matplotlib import pyplot as plt

# Functions

In [6]:
def read_texts(path):
    data = []
    file_name = os.listdir(path)

    for name in file_name:
        if name.endswith('.txt'):
            with open(path + name,encoding="utf8") as f:
                text = f.read()
                data.append({'nombre':name.replace('.txt',''), 'texto':text})

    df = pd.DataFrame(data)
    return df

In [7]:
def clean_text(string):
    """
    A method to clean text 
    """
    
    # Removing the punctuations
    for x in string.lower(): 
        if x in punctuation:
            if x != '/':
                string = string.replace(x, "")
            else:
                string = string.replace(x, " ")
    
    string = unidecode.unidecode(string)

#     # Converting the text to lower
#     string = string.lower()

    # Removing stop words
    string = ' '.join([word for word in string.split() if word not in swords])

    # Cleaning the whitespaces
    string = re.sub(r'\s+', ' ', string).strip()

    return string 

In [8]:
#nltk.download('stopwords')
swords = list(set(stopwords.words('spanish')))

In [9]:
def sep_num_text(data):

    words = word_tokenize(data) 
    for w in words:
        if re.search(r'\d', w):
            ind = words.index(w)
            words[ind] = [''.join(g) for k, g in itertools.groupby(w, str.isalpha)]
    data = ' '.join([x if type(x) is not list else ' '.join(x) for x in words])
    
    return data

In [10]:
def get_f_b_context_text(data):
    
    '''Return context in a string format'''
    
    all_data = []
    sf_not_found = []
    
    for instance in data:
           
        texto = instance['texto']    
        target_word = instance['short_form'] 
        
        if target_word in texto:

            stop_ini_idx = instance['StartOffset'] #índice del inicio de la target
            stop_fin_idx = instance['EndOffset'] #índice del inicio de la target

            _instance = []
            xf = texto[:stop_ini_idx] + ' <start> ' +texto[stop_ini_idx:stop_fin_idx] + ' <end> ' #palabras anteriores a la target
            xb = texto[stop_fin_idx+1:]   #palabras posteriores a la target            

            instance_id = instance['long_form'] #id del significado

            _instance.append(target_word)
            _instance.append(xf)
            _instance.append(xb)
            _instance.append(instance_id)

            all_data.append(_instance[:])
        else:
            sf_not_found.append(target_word)
#             print("El acrónimo {} no aparece en el texto {}".format(target_word, instance['doc_id']))
        
    return all_data, sf_not_found

In [11]:
def limit_context(data):

    for doc in data:
        
        sf, xf, xb, lf = doc[0], doc[1], doc[2], doc[3]

        xf_words = word_tokenize(xf)[-n_step_f:]
        xb_words = word_tokenize(xb)[:n_step_b]

        doc[1] = ' '.join(xf_words)
        doc[2] = ' '.join(xb_words)   

    return data        

In [12]:
def create_dict(data):
    
    data_dic = []
    
    for instance in data:
            
        dic = {}

        dic['short_form'] = instance[0]
        dic['context'] = instance[1] + ' ' + instance[2]
        dic['long_form'] = instance[3]
    
        data_dic.append(dic)
        
    return data_dic
    

# Load Data

### Testing

220 clinical cases.

In [13]:
testing_abbr = pd.read_csv("../datasets/testing_set/clinical_cases.abbreviations.testing_set.tsv", sep = '\t')
testing_met = pd.read_csv("../datasets/testing_set/clinical_cases.metadata.testing_set.tsv", sep = '\t')
testing_rel = pd.read_csv("../datasets/testing_set/clinical_cases.relations.testing_set.tsv", sep = '\t')

In [14]:
testing_met.head()

,# Document_ID,Case_ID,ISSN,Date,Source,Full_Text_Link
0,S1130-01082006000700014-1.txt,1.txt,1130-0108,2006-07-01,Revista Española de Enfermedades Digestivas v.98 n.7 2006,http://scielo.isciii.es/scielo.php?script=sci_arttext&pid=S1130-01082006000700014
1,S1130-01082007000300006-7.txt,7.txt,1130-0108,2007-03-01,Revista Española de Enfermedades Digestivas v.99 n.3 2007,http://scielo.isciii.es/scielo.php?script=sci_arttext&pid=S1130-01082007000300006
2,S1134-80462009000100005-1.txt,1.txt,1134-8046,2009-02-01,Revista de la Sociedad Española del Dolor v.16 n.1 2009,http://scielo.isciii.es/scielo.php?script=sci_arttext&pid=S1134-80462009000100005
3,S1137-66272014000300015-1.txt,1.txt,1137-6627,2014-12-01,Anales del Sistema Sanitario de Navarra v.37 n.3 2014,http://scielo.isciii.es/scielo.php?script=sci_arttext&pid=S1137-66272014000300015
4,S0365-66912004001200011-1.txt,1.txt,0365-6691,2004-12-01,Archivos de la Sociedad Española de Oftalmología v.79 n.12 2004,http://scielo.isciii.es/scielo.php?script=sci_arttext&pid=S0365-66912004001200011


In [15]:
testing_rel = testing_rel.reset_index()

In [16]:
testing_rel.columns = ['# Document_ID', 'Mention_A_type', 'Mention_A_StartOffset',
      'Mention_A', 'Relation_type', 'Mention_B_type',
       'Mention_B_StartOffset', 'Mention_B_EndOffset', 'Mention_B']

In [17]:
testing_rel = testing_rel.rename(columns = {'# Document_ID': 'doc_id'})

In [18]:
testing_rel.head()

,doc_id,Mention_A_type,Mention_A_StartOffset,Mention_A,Relation_type,Mention_B_type,Mention_B_StartOffset,Mention_B_EndOffset,Mention_B
0,S0211-69952013000500019-1,SHORT_FORM,3739,ARA II,SHORT-LONG,LONG_FORM,3695,3737,antagonista del receptor de angiotesina II
1,S0211-69952013000500019-1,SHORT_FORM,2793,PCR,SHORT-LONG,LONG_FORM,2798,2837,reacción en cadena de enzima polimerasa
2,S0365-66912004000600008-1,SHORT_FORM,406,AV,SHORT-LONG,LONG_FORM,390,404,agudeza visual
3,S0211-69952012000500025-1,SHORT_FORM,945,angio-TAC,SHORT-LONG,LONG_FORM,908,943,angiotomografía computarizada axial
4,S1130-05582017000200122-1,SHORT_FORM,940,RMN,SHORT-LONG,LONG_FORM,910,938,resonancia magnética nuclear


In [19]:
testing_rel.Relation_type.unique()

array(['SHORT-LONG', 'SHORT-NESTED', 'NESTED-LONG'], dtype=object)

In [20]:
testing_abbr = testing_abbr.rename(columns = {'# Document_ID': 'doc_id'})

In [21]:
testing_abbr.head()

,doc_id,StartOffset,EndOffset,Abbreviation,Definition,Definition_lemmatized
0,S0004-06142010000500014-1,2037,2044,16SrRNA,16s ribosomal rna,16s ribosomal rno
1,S0004-06142010000500014-1,1349,1351,M.,mycobacterium,mycobacterium
2,S0004-06142010000500014-1,1339,1342,PCR,polymerase chain reaction,polymerase chain reaction
3,S0004-06142010000500014-1,611,615,BHCG,beta-human chorionic gonadotropin,beta-humar chorionic gonadotropin
4,S0004-06142010000500014-1,594,597,CEA,carcinoembrionary antigen,carcinoembrionary antiger


In [22]:
testing_raw = read_texts("../datasets/testing_set/testing_set.raw_text/")

In [23]:
testing_raw.head()

,nombre,texto
0,S1139-76322009000700016-1,Paciente de sexo femenino de 13 años y 7 meses de edad que consulta por dolor abdominal y vómito...
1,S0210-48062007000700015-1,"Varón de 72 años con antecedentes personales de hipertensión arterial, enfermedad pulmonar obstr..."
2,S0212-71992005000500009-1,"Varón de 81 años, con antecedentes de fibrilación auricular crónica, que acudió a nuestro hospit..."
3,S0365-66912004001200011-1,Paciente varón de 52 años que acudió a urgencias por pérdida de visión en su ojo derecho de una ...
4,S1130-01082009000900015-1,"Varón de 54 años con episodios de pancreatitis aguda de repetición, que como antecedentes person..."


### Development

146 clinical cases

In [24]:
dev_abbr = pd.read_csv("../datasets/development_set/clinical_cases.abbreviations.development_set.tsv", sep = '\t')
dev_met = pd.read_csv("../datasets/development_set/clinical_cases.metadata.development_set.tsv", sep = '\t')
dev_rel = pd.read_csv("../datasets/development_set/clinical_cases.relations.development_set.tsv", sep = '\t')

In [25]:
dev_met = dev_met.rename(columns = {'# Document_ID': 'doc_id'})

In [26]:
dev_met.head()

,doc_id,Case_ID,ISSN,Date,Source,Full_Text_Link
0,S1130-14732005000300004-1.txt,1.txt,1130-1473,2005-06-01,Neurocirugía v.16 n.3 2005,http://scielo.isciii.es/scielo.php?script=sci_arttext&pid=S1130-14732005000300004
1,S1130-01082008000200009-1.txt,1.txt,1130-0108,2008-02-01,Revista Española de Enfermedades Digestivas v.100 n.2 2008,http://scielo.isciii.es/scielo.php?script=sci_arttext&pid=S1130-01082008000200009
2,S1137-66272012000300021-1.txt,1.txt,1137-6627,2012-12-01,Anales del Sistema Sanitario de Navarra v.35 n.3 2012,http://scielo.isciii.es/scielo.php?script=sci_arttext&pid=S1137-66272012000300021
3,S1699-695X2016000200010-1.txt,1.txt,1699-695X,2016-06-01,Revista Clínica de Medicina de Familia v.9 n.2 2016,http://scielo.isciii.es/scielo.php?script=sci_arttext&pid=S1699-695X2016000200010
4,S1130-01082007001100009-1.txt,1.txt,1130-0108,2007-11-01,Revista Española de Enfermedades Digestivas v.99 n.11 2007,http://scielo.isciii.es/scielo.php?script=sci_arttext&pid=S1130-01082007001100009


In [27]:
dev_rel = dev_rel.reset_index()

In [28]:
dev_rel.columns = ['# Document_ID', 'Mention_A_type', 'Mention_A_StartOffset',
      'Mention_A', 'Relation_type', 'Mention_B_type',
       'Mention_B_StartOffset', 'Mention_B_EndOffset', 'Mention_B']

In [29]:
dev_rel = dev_rel.rename(columns = {'# Document_ID': 'doc_id'})

In [30]:
dev_rel.head()

,doc_id,Mention_A_type,Mention_A_StartOffset,Mention_A,Relation_type,Mention_B_type,Mention_B_StartOffset,Mention_B_EndOffset,Mention_B
0,S1888-75462015000400006-1,SHORT_FORM,1436,AP,SHORT-LONG,LONG_FORM,1419,1434,anteroposterior
1,S0210-56912009000700006-2,SHORT_FORM,876,angio-RMN,SHORT-LONG,LONG_FORM,858,874,angiorresonancia
2,S0210-56912009000700006-2,SHORT_FORM,819,RMN,SHORT-LONG,LONG_FORM,789,817,resonancia magnética nuclear
3,S1698-44472005000300015-1,SHORT_FORM,209,ATM,SHORT-LONG,LONG_FORM,177,207,articulación témporomandibular
4,S1698-44472005000300015-1,SHORT_FORM,1125,TC,SHORT-LONG,LONG_FORM,1099,1123,tomografía computerizada


In [31]:
dev_abbr = dev_abbr.rename(columns = {'# Document_ID': 'doc_id'})

In [32]:
dev_abbr.head()

,doc_id,StartOffset,EndOffset,Abbreviation,Definition,Definition_lemmatized
0,S1130-14732005000300004-1,1216,1218,C3,tercera vértebra cervical,tercero vértebra cervical
1,S1130-14732005000300004-1,717,719,C2,segunda vértebra cervical,segundo vértebra cervical
2,S1130-14732005000300004-1,3191,3193,C3,tercera vértebra cervical,tercero vértebra cervical
3,S1130-14732005000300004-1,2867,2869,C3,tercera vértebra cervical,tercero vértebra cervical
4,S1130-14732005000300004-1,2862,2864,C2,segunda vértebra cervical,segundo vértebra cervical


In [33]:
dev_raw = read_texts("../datasets/development_set/development_set.raw_text/")

In [34]:
dev_raw.head()

,nombre,texto
0,S1139-76322017000200010-1,Niña de dos años y diez meses con antecedentes de dermatitis atópica que acude a urgencias por l...
1,S0365-66912005001100008-1,"Se presenta el caso de un varón de 45 años que acude por dolor ocular bilateral, fotofobia inten..."
2,S1130-01082006001000017-1,"Mujer de 42 años de edad con antecedentes personales de hernia umbilical intervenida, HTA. Prese..."
3,S0212-71992005001200008-1,Paciente de 57 años con ingresos hospitalarios repetidos por epigastralgia acompañadas de elevac...
4,S0365-66912007000300010-1,Paciente de 33 años que el 20-08-05 es trasladado a urgencias por un corte sangrante en el cuell...


### Sample test

15 clinical cases

In [35]:
sample_abbr = pd.read_csv("../datasets/sample_set/clinical_cases.abbreviations.sample_set.tsv", sep = '\t')
sample_met = pd.read_csv("../datasets/sample_set/clinical_cases.metadata.sample_set.tsv", sep = '\t')
sample_rel = pd.read_csv("../datasets/sample_set/clinical_cases.relations.sample_set.tsv", sep = '\t')

In [36]:
sample_met = sample_met.rename(columns = {'# Document_ID': 'doc_id'})

In [37]:
sample_met.head()

,doc_id,Case_ID,ISSN,Date,Source,Full_Text_Link
0,S0004-06142006000900015-1,1,0004-0614,2006-11-01,Archivos Españoles de Urología (Ed. impresa) v.59 n.9 2006,http://scielo.isciii.es/scielo.php?script=sci_arttext&pid=S0004-06142006000900015
1,S0004-06142006000600015-1,1,0004-0614,2006-08-01,Archivos Españoles de Urología (Ed. impresa) v.59 n.6 2006,http://scielo.isciii.es/scielo.php?script=sci_arttext&pid=S0004-06142006000600015
2,S0004-06142007000700014-1,1,0004-0614,2007-09-01,Archivos Españoles de Urología (Ed. impresa) v.60 n.7 2007,http://scielo.isciii.es/scielo.php?script=sci_arttext&pid=S0004-06142007000700014
3,S0004-06142007000900013-1,1,0004-0614,2007-11-01,Archivos Españoles de Urología (Ed. impresa) v.60 n.9 2007,http://scielo.isciii.es/scielo.php?script=sci_arttext&pid=S0004-06142007000900013
4,S0004-06142006000200014-1,1,0004-0614,2006-03-01,Archivos Españoles de Urología (Ed. impresa) v.59 n.2 2006,http://scielo.isciii.es/scielo.php?script=sci_arttext&pid=S0004-06142006000200014


In [38]:
sample_rel = sample_rel.rename(columns = {'# Document_ID': 'doc_id'})

In [39]:
sample_rel.head()

,doc_id,Mention_A_type,Mention_A_StartOffset,Mention_A_EndOffset,Mention_A,Relation_type,Mention_B_type,Mention_B_StartOffset,Mention_B_EndOffset,Mention_B
0,S0004-06142006000700014-1,SHORT_FORM,926,929,CEA,SHORT-LONG,LONG_FORM,896,924,Antígeno Carcino Embrionario
1,S0004-06142005001000011-1,SHORT_FORM,1626,1629,ROT,SHORT-LONG,LONG_FORM,1600,1624,reflejos osteotendinosos
2,S0004-06142005001000011-1,SHORT_FORM,1715,1718,RMN,SHORT-LONG,LONG_FORM,1685,1713,resonancia magnética nuclear
3,S0004-06142005001000011-1,SHORT_FORM,1663,1666,RCP,SHORT-LONG,LONG_FORM,1639,1661,reflejo cutaneoplantar
4,S0004-06142005001000011-1,SHORT_FORM,1808,1811,LCR,SHORT-LONG,LONG_FORM,1783,1806,líquido cefalorraquídeo


In [40]:
sample_abbr = sample_abbr.rename(columns = {'# Document_ID': 'doc_id'})

In [41]:
sample_abbr.head()

,doc_id,StartOffset,EndOffset,Abbreviation,Definition,Definition_lemmatized
0,S0004-06142005001000011-1,1034,1036,Kg,kilogramo,kilogramo
1,S0004-06142005001000011-1,1031,1033,mg,miligramo,miligramo
2,S0004-06142005001000011-1,196,199,IgA,inmunoglobulina a,inmunoglobulina a
3,S0004-06142005001000011-1,2057,2060,LCR,líquido cefalorraquídeo,líquido cefalorraquídeo
4,S0004-06142005001000011-1,1594,1598,EEII,extremidades inferiores,extremidad inferior


In [42]:
sample_raw = read_texts("../datasets/sample_set/sample_set.raw_text/")

In [43]:
sample_raw.head()

,nombre,texto
0,S0004-06142006000600014-1,"Paciente varón, de 40 años de edad, con antecedentes de salud. Acude a consulta de Urología por ..."
1,S0004-06142006000300015-1,Paciente de 50 años con antecedente de litiasis renal de repetición que consultó por hematuria r...
2,S0004-06142007000900013-1,"Presentamos el caso de un recién nacido de tres días de vida, que acude a urgencias con su madre..."
3,S0004-06142005001000011-1,"Varón de 58 años de edad en el momento del trasplante, el 5 de octubre de 1998.\nEn programa de ..."
4,S0004-06142006000700013-1,"Paciente varón, de 63 años de edad, mestizo, diabético y fumador inveterado. Se presentó a consu..."


### Trainning

318 clinical cases

In [44]:
train_abbr = pd.read_csv("../datasets/trainning_set/clinical_cases.abbreviations.training_set.tsv", sep = '\t')

In [45]:
train_met = pd.read_csv("../datasets/trainning_set/clinical_cases.metadata.training_set.tsv", sep = '\t')
train_rel = pd.read_csv("../datasets/trainning_set/clinical_cases.relations.training_set.tsv", sep = '\t')

In [46]:
train_met = train_met.rename(columns = {'# Document_ID': 'doc_id'})

In [47]:
train_met.head()

,doc_id,Case_ID,ISSN,Date,Source,Full_Text_Link
0,S1139-76322015000500009-1.txt,1.txt,1139-7632,2015-12-01,Pediatría Atención Primaria v.17 n.68 2015,http://scielo.isciii.es/scielo.php?script=sci_arttext&pid=S1139-76322015000500009
1,S1130-05582008000400007-2.txt,2.txt,1130-0558,2008-08-01,Revista Española de Cirugía Oral y Maxilofacial v.30 n.4 2008,http://scielo.isciii.es/scielo.php?script=sci_arttext&pid=S1130-05582008000400007
2,S0210-48062006000100012-1.txt,1.txt,0210-4806,2006-01-01,Actas Urológicas Españolas v.30 n.1 2006,http://scielo.isciii.es/scielo.php?script=sci_arttext&pid=S0210-48062006000100012
3,S0213-12852003000500002-1.txt,1.txt,0213-1285,2003-10-01,Avances en Odontoestomatología v.19 n.5 2003,http://scielo.isciii.es/scielo.php?script=sci_arttext&pid=S0213-12852003000500002
4,S0212-71992005000400007-1.txt,1.txt,0212-7199,2005-04-01,Anales de Medicina Interna v.22 n.4 2005,http://scielo.isciii.es/scielo.php?script=sci_arttext&pid=S0212-71992005000400007


In [48]:
train_rel = train_rel.reset_index()

In [49]:
train_rel.columns = ['# Document_ID', 'Mention_A_type', 'Mention_A_StartOffset',
      'Mention_A', 'Relation_type', 'Mention_B_type',
       'Mention_B_StartOffset', 'Mention_B_EndOffset', 'Mention_B']

In [50]:
train_rel = train_rel.rename(columns = {'# Document_ID': 'doc_id'})

In [51]:
train_rel.head()

,doc_id,Mention_A_type,Mention_A_StartOffset,Mention_A,Relation_type,Mention_B_type,Mention_B_StartOffset,Mention_B_EndOffset,Mention_B
0,S1130-01082009000400014-1,SHORT_FORM,476,NPT,SHORT-LONG,LONG_FORM,454.0,474.0,nutrición parenteral
1,S1130-63432016000100009-1,SHORT_FORM,614,NIHSS,SHORT-LONG,LONG_FORM,621.0,662.0,National Institute of Health Stroke Scale
2,S1139-76322017000200007-1,SHORT_FORM,1145,CMV,SHORT-LONG,LONG_FORM,1128.0,1143.0,citomegalovirus
3,S1139-76322017000200007-1,SHORT_FORM,1243,VSG,SHORT-LONG,LONG_FORM,1206.0,1241.0,velocidad de sedimentación globular
4,S1139-76322017000200007-1,SHORT_FORM,1300,IGRA,SHORT-LONG,LONG_FORM,1267.0,1298.0,interferon-gamma release assays


In [52]:
train_abbr = train_abbr.rename(columns = {'# Document_ID': 'doc_id'})

In [53]:
train_abbr.Definition.nunique()

908

In [54]:
train_abbr.head()

,doc_id,StartOffset,EndOffset,Abbreviation,Definition,Definition_lemmatized
0,S0210-48062004000500008-1,1650,1652,ml,mililitro,mililitro
1,S0210-48062004000500008-1,708,709,l,litro,litro
2,S0210-48062004000500008-1,704,707,mEq,miliequivalente,miliequivalente
3,S0210-48062004000500008-1,677,681,pCO2,presión parcial de co2,presión parcial de co2
4,S0210-48062004000500008-1,2287,2290,HLA,human leucocyte antigen,human leucocyte antiger


In [55]:
train_raw = read_texts("../datasets/trainning_set/training_set.raw_text/")

In [56]:
train_raw = train_raw.rename(columns = {'nombre': 'doc_id'})

In [57]:
train_raw.head()

,doc_id,texto
0,S1130-05582012000300005-1,Acude a nuestras consultas a un paciente que presenta una tumoración cervical izquierda de gran ...
1,S0212-71992005000400009-1,"Se trataba de un varón de 27 años de edad, que había sufrido una neumonía siendo niño y que no e..."
2,S0004-06142008000700015-2,Varón de 33 años fumador de un paquete de cigarrillos al día sin otros antecedentes de interés q...
3,S0210-56912006000800008-1,"Hombre de 42 años, bebedor de más de 100 g de etanol al día, sin otros antecedentes de interés, ..."
4,S0376-78922009000300010-1,Paciente de 18 años de edad que 5 meses antes había sido víctima de grave quemadura eléctrica po...


## Prepare Train Data

### Preprocessing

In [58]:
train = train_raw.merge(train_abbr[['doc_id', 'Abbreviation', 'Definition_lemmatized']], on = 'doc_id', how = 'left')

In [59]:
train = train.merge(train_abbr, on = ['doc_id', 'Abbreviation'], how = 'left')

In [60]:
train = train.drop_duplicates()

In [61]:
train = train[['doc_id', 'texto', 'Abbreviation', 'Definition_lemmatized_x', 'StartOffset', 'EndOffset']]

In [62]:
train = train.rename(columns = {'Definition_lemmatized_x':'Definition'})

In [63]:
train.head()

,doc_id,texto,Abbreviation,Definition,StartOffset,EndOffset
0,S1130-05582012000300005-1,Acude a nuestras consultas a un paciente que presenta una tumoración cervical izquierda de gran ...,PAAF,punción aspiración con aguja fino,1006.0,1010.0
1,S1130-05582012000300005-1,Acude a nuestras consultas a un paciente que presenta una tumoración cervical izquierda de gran ...,RM,resonancia magnético,789.0,791.0
2,S1130-05582012000300005-1,Acude a nuestras consultas a un paciente que presenta una tumoración cervical izquierda de gran ...,cm,centímetro,1345.0,1347.0
3,S1130-05582012000300005-1,Acude a nuestras consultas a un paciente que presenta una tumoración cervical izquierda de gran ...,cm,centímetro,1217.0,1219.0
4,S1130-05582012000300005-1,Acude a nuestras consultas a un paciente que presenta una tumoración cervical izquierda de gran ...,g,gramo,1304.0,1305.0


In [64]:
train = train.rename(columns = {'Abbreviation': 'short_form', 'Definition': 'long_form'})

In [65]:
train = train.dropna(subset = ['short_form', 'long_form'])

In [66]:
train[['StartOffset', 'EndOffset']] = train[['StartOffset', 'EndOffset']].astype(int)

Clean long forms to delete acents, string punctuation etc

In [67]:
train['long_form'] = train['long_form'].map(clean_text)

Transform dataframe to a list of dictionaries

In [68]:
train_data = train[['doc_id', 'texto', 'short_form', 'long_form', 'StartOffset', 'EndOffset']].to_dict('records')

### Get the text before and after the SF

In [69]:
train_ndata, sf_not_found = get_f_b_context_text(train_data)

Check if any SF is not founded in the text

In [70]:
sf_not_found_set = set(sf_not_found)

In [71]:
len(sf_not_found_set)

0

Now clean the texts, remove string punctuation, accents, lower case, remove double spaces, separate numbers from sf

In [72]:
# def clean_text_after(data):
    
#     for instance in data:
#         instance[1] = clean_text(instance[1]).lower()
#         instance[2] = clean_text(instance[2]).lower()
#         instance[1] = re.sub(r'(\d+)', r'\g<1> ', instance[1]) #metemos espacio entre número y acrónimos que quedan pegados    
#         instance[2] = re.sub(r'(\d+)', r'\g<1> ', instance[2])
#         instance[1] = re.sub(r'\s+', ' ', instance[1]).strip()
#         instance[2] = re.sub(r'\s+', ' ', instance[2]).strip()
#     return data
        

In [73]:
# train_ndata = clean_text_after(train_ndata)

### Get the contexts 

From the texts before and after the sf, we limit the number of words

In [74]:
n_step_f = 30 #number of words to select from the forward context
n_step_b = 30#number of words to select from the backward context

In [75]:
train_ndata = limit_context(train_ndata)

In [76]:
train_ndata[0]

['PAAF',
 'no invadía estructuras vecinas y con una densidad grasa . Se realiza la toma de una muestra mediante punción-aspiración con aguja fina ( < start > PAAF < end >',
 'que confirmó las sospechas iniciales de lipoma . Se decidió realizar escisión quirúrgica de la tumoración . Se realizó un abordaje a nivel de la línea posterior del cabello ,',
 'puncion aspiracion aguja fino']

### Transform into df grouped by LF

In [77]:
data_dic = create_dict(train_ndata)

In [78]:
data_df = pd.DataFrame(data_dic)

Asign an id to each LF

In [79]:
data_df = data_df.assign(id=(data_df['long_form']).astype('category').cat.codes)

In [80]:
data_df.head()

,short_form,context,long_form,id
0,PAAF,no invadía estructuras vecinas y con una densidad grasa . Se realiza la toma de una muestra medi...,puncion aspiracion aguja fino,597
1,RM,y la exploración intraoral fue normal . El resto de la exploración física fue completamente anod...,resonancia magnetico,625
2,cm,"5*10cm para mejorar el cierre directo . Durante la intervención se obtuvo una pieza de 459,5g de...",centimetro,125
3,cm,"de la tumoración . Se realizó un abordaje a nivel de la línea posterior del cabello , resecando ...",centimetro,125
4,g,"cabello , resecando un huso cutáneo de 5*10cm para mejorar el cierre directo . Durante la interv...",gramo,328


In [81]:
data_df.shape

(4355, 4)

Vamos a unir cada SF con todas las posibles LF, una por cada registro, para que posteriormente el modelo funcione de forma binaria asignando la probabilidad de que ese sea su LF

In [82]:
sf_lf_unique = data_df[['short_form', 'long_form']].drop_duplicates()

In [83]:
sf_lf_unique.shape

(871, 2)

In [84]:
sf_lf_unique.sort_values('short_form').head()

,short_form,long_form
1742,5-FU,5fluorouracilo
899,67Ga,gallium67
3355,99-MDP,99 metilendifosfonato
901,99Tc,tecnecio99
669,A,antigeno leucocitario humano


In [95]:
data_merged = data_df.merge(sf_lf_unique, on = 'short_form', how = 'left')

In [96]:
data_merged.shape

(6360, 5)

In [99]:
pd.set_option('display.max_colwidth', 100)

In [100]:
data_df.sort_values(['short_form', 'context']).head()

,short_form,context,long_form,id
1743,5-FU,"% . En la exploración física , cabe destacar una marcada pigmentación cutánea generalizada y alo...",5fluorouracilo,1
1744,5-FU,"con esquema mFOLFOX 6 + Bevacizumab ( oxaliplatino 85 mg/m2 i.v . día 1 , leucovorin 200 mg/m2 i...",5fluorouracilo,1
1742,5-FU,"i.v . día 1 , leucovorin 200 mg/m2 i.v . día 1 , 5-FU 400 mg/m2 en bolo i.v . día 1 , < start > ...",5fluorouracilo,1
899,67Ga,trocánter izquierdo . La radiología de pelvis mostró una única lesión lítica en la cabeza femora...,gallium67,303
3355,99-MDP,"resección hepática parcial , al mes de su diagnostico . Previamente se había realizado un estudi...",99 metilendifosfonato,2


In [103]:
data_merged.sort_values(['short_form', 'context']).head()

,short_form,context,long_form_x,id,long_form_y
2640,5-FU,"% . En la exploración física , cabe destacar una marcada pigmentación cutánea generalizada y alo...",5fluorouracilo,1,5fluorouracilo
2641,5-FU,"con esquema mFOLFOX 6 + Bevacizumab ( oxaliplatino 85 mg/m2 i.v . día 1 , leucovorin 200 mg/m2 i...",5fluorouracilo,1,5fluorouracilo
2639,5-FU,"i.v . día 1 , leucovorin 200 mg/m2 i.v . día 1 , 5-FU 400 mg/m2 en bolo i.v . día 1 , < start > ...",5fluorouracilo,1,5fluorouracilo
1336,67Ga,trocánter izquierdo . La radiología de pelvis mostró una única lesión lítica en la cabeza femora...,gallium67,303,gallium67
4944,99-MDP,"resección hepática parcial , al mes de su diagnostico . Previamente se había realizado un estudi...",99 metilendifosfonato,2,99 metilendifosfonato


El label debe de ser 0 o 1, es decir, si la LF que le corresponde es la que se le asigna. Por ello creamos una etiqueta binaria.

In [104]:
def get_label(row):
    if row['long_form_x'] == row['long_form_y']:
        return 1
    else:
        return 0

In [105]:
data_merged['label'] = data_merged.apply(get_label, axis = 1)

In [107]:
data_merged.head()

,short_form,context,long_form_x,id,long_form_y,label
0,PAAF,no invadía estructuras vecinas y con una densidad grasa . Se realiza la toma de una muestra medi...,puncion aspiracion aguja fino,597,puncion aspiracion aguja fino,1
1,PAAF,no invadía estructuras vecinas y con una densidad grasa . Se realiza la toma de una muestra medi...,puncion aspiracion aguja fino,597,puncion aspirativo aguja fino,0
2,RM,y la exploración intraoral fue normal . El resto de la exploración física fue completamente anod...,resonancia magnetico,625,resonancia magnetico,1
3,cm,"5*10cm para mejorar el cierre directo . Durante la intervención se obtuvo una pieza de 459,5g de...",centimetro,125,centimetro,1
4,cm,"5*10cm para mejorar el cierre directo . Durante la intervención se obtuvo una pieza de 459,5g de...",centimetro,125,centimetro agua,0


In [108]:
data_merged = data_merged[['short_form', 'context', 'long_form_y', 'label']]

In [109]:
data_merged = data_merged.rename(columns = {'long_form_y':'long_form'})

In [110]:
data_merged.reset_index(drop = True, inplace = True)

In [111]:
data_merged.head()

,short_form,context,long_form,label
0,PAAF,no invadía estructuras vecinas y con una densidad grasa . Se realiza la toma de una muestra medi...,puncion aspiracion aguja fino,1
1,PAAF,no invadía estructuras vecinas y con una densidad grasa . Se realiza la toma de una muestra medi...,puncion aspirativo aguja fino,0
2,RM,y la exploración intraoral fue normal . El resto de la exploración física fue completamente anod...,resonancia magnetico,1
3,cm,"5*10cm para mejorar el cierre directo . Durante la intervención se obtuvo una pieza de 459,5g de...",centimetro,1
4,cm,"5*10cm para mejorar el cierre directo . Durante la intervención se obtuvo una pieza de 459,5g de...",centimetro agua,0


In [112]:
data_merged.to_csv('train_data_beto.csv')

## Prepare Test Data

### Preprocessing

In [65]:
testing_raw.rename(columns={'nombre':'doc_id'}, inplace = True)

In [66]:
test = testing_raw.merge(testing_abbr[['doc_id', 'Abbreviation', 'Definition_lemmatized']], on = 'doc_id', how = 'left')

In [68]:
test = test.merge(testing_abbr, on = ['doc_id', 'Abbreviation'], how = 'left')

In [69]:
test = test.drop_duplicates()

In [70]:
test = test[['doc_id', 'texto', 'Abbreviation', 'Definition_lemmatized_x', 'StartOffset', 'EndOffset']]

In [71]:
test = test.rename(columns = {'Definition_lemmatized_x':'Definition'})

In [72]:
test.head()

,doc_id,texto,Abbreviation,Definition,StartOffset,EndOffset
0,S1139-76322009000700016-1,Paciente de sexo femenino de 13 años y 7 meses de edad que consulta por dolor abdominal y vómito...,mm,milímetro,1901.0,1903.0
1,S1139-76322009000700016-1,Paciente de sexo femenino de 13 años y 7 meses de edad que consulta por dolor abdominal y vómito...,mg,miligramo,1627.0,1629.0
2,S1139-76322009000700016-1,Paciente de sexo femenino de 13 años y 7 meses de edad que consulta por dolor abdominal y vómito...,mg,miligramo,1737.0,1739.0
3,S1139-76322009000700016-1,Paciente de sexo femenino de 13 años y 7 meses de edad que consulta por dolor abdominal y vómito...,mg,miligramo,1715.0,1717.0
4,S1139-76322009000700016-1,Paciente de sexo femenino de 13 años y 7 meses de edad que consulta por dolor abdominal y vómito...,mg,miligramo,1650.0,1652.0


In [73]:
test = test.rename(columns = {'Abbreviation': 'short_form', 'Definition': 'long_form'})

In [74]:
test = test.dropna(subset = ['short_form', 'long_form'])

In [75]:
test[['StartOffset', 'EndOffset']] = test[['StartOffset', 'EndOffset']].astype(int)

Clean long forms to delete acents, string punctuation etc

In [76]:
test['long_form'] = test['long_form'].map(clean_text)

Transform dataframe to a list of dictionaries

In [77]:
test_data = test[['doc_id', 'texto', 'short_form', 'long_form', 'StartOffset', 'EndOffset']].to_dict('records')

### Get the text before and after the SF

In [78]:
test_ndata, sf_not_found = get_f_b_context_text(test_data)

Check if any SF is not founded in the text

In [79]:
sf_not_found_set = set(sf_not_found)

In [80]:
len(sf_not_found_set)

0

Now clean the texts, remove string punctuation, accents, lower case, remove double spaces, separate numbers from sf

In [72]:
# def clean_text_after(data):
    
#     for instance in data:
#         instance[1] = clean_text(instance[1]).lower()
#         instance[2] = clean_text(instance[2]).lower()
#         instance[1] = re.sub(r'(\d+)', r'\g<1> ', instance[1]) #metemos espacio entre número y acrónimos que quedan pegados    
#         instance[2] = re.sub(r'(\d+)', r'\g<1> ', instance[2])
#         instance[1] = re.sub(r'\s+', ' ', instance[1]).strip()
#         instance[2] = re.sub(r'\s+', ' ', instance[2]).strip()
#     return data
        

In [73]:
# train_ndata = clean_text_after(train_ndata)

### Get the contexts 

From the texts before and after the sf, we limit the number of words

In [82]:
n_step_f = 30 #number of words to select from the forward context
n_step_b = 30#number of words to select from the backward context

In [83]:
test_ndata = limit_context(test_ndata)

In [84]:
test_ndata[0]

['mm',
 '243 UI/L , amilasas 427,8 UI/L , lipasa 1298UI/L . Ecografía abdominal : vía biliar poco distendida con imagen de litiasis de 15 < start > mm < end >',
 'Páncreas no valorable por interposición de gas intestinal . Diagnóstico final y evolución : se trata pues , de un caso de pancreatitis aguda secundaria a litiasis biliar . Se',
 'milimetro']

### Transform into df grouped by LF

In [85]:
data_dic = create_dict(test_ndata)

In [86]:
data_df = pd.DataFrame(data_dic)

Asign an id to each LF

In [79]:
data_df = data_df.assign(id=(data_df['long_form']).astype('category').cat.codes)

In [80]:
data_df.head()

,short_form,context,long_form,id
0,PAAF,no invadía estructuras vecinas y con una densidad grasa . Se realiza la toma de una muestra medi...,puncion aspiracion aguja fino,597
1,RM,y la exploración intraoral fue normal . El resto de la exploración física fue completamente anod...,resonancia magnetico,625
2,cm,"5*10cm para mejorar el cierre directo . Durante la intervención se obtuvo una pieza de 459,5g de...",centimetro,125
3,cm,"de la tumoración . Se realizó un abordaje a nivel de la línea posterior del cabello , resecando ...",centimetro,125
4,g,"cabello , resecando un huso cutáneo de 5*10cm para mejorar el cierre directo . Durante la interv...",gramo,328


In [81]:
data_df.shape

(4355, 4)

Vamos a unir cada SF con todas las posibles LF, una por cada registro, para que posteriormente el modelo funcione de forma binaria asignando la probabilidad de que ese sea su LF

In [87]:
sf_lf_unique = data_df[['short_form', 'long_form']].drop_duplicates()

In [88]:
sf_lf_unique.shape

(780, 2)

In [89]:
sf_lf_unique.sort_values('short_form').head()

,short_form,long_form
42,/mm3,milimetro cubico
1837,16SrRNA,16s ribosomal rno
831,3D,3 dimension
1203,5-ASA,5aminosalicylic acid
351,A,adeniner


In [90]:
data_merged = data_df.merge(sf_lf_unique, on = 'short_form', how = 'left')

In [91]:
data_merged.shape

(4703, 4)

In [92]:
pd.set_option('display.max_colwidth', 100)

In [93]:
data_df.sort_values(['short_form', 'context']).head()

,short_form,context,long_form
42,/mm3,de parénquima pulmonar . El análisis de sangre fue normal . La toracocentesis mostró un líquido ...,milimetro cubico
1837,16SrRNA,Neelsen modificada fue negativa . La identificación fenotípica no aportaba datos significativos ...,16s ribosomal rno
831,3D,"externa , mediante Acelerador Lineal con fotones de 6 y 15 MV , con cuñas de compensación tisula...",3 dimension
1203,5-ASA,"plasmáticas . Ante la sospecha inicial , dada la clínica y los hallazgos endoscópicos , de colit...",5aminosalicylic acid
2694,A,"V y 20210 del factor II , realizándose posteriormente una electroforesis que identificó la mutac...",adenina


In [103]:
data_merged.sort_values(['short_form', 'context']).head()

,short_form,context,long_form_x,id,long_form_y
2640,5-FU,"% . En la exploración física , cabe destacar una marcada pigmentación cutánea generalizada y alo...",5fluorouracilo,1,5fluorouracilo
2641,5-FU,"con esquema mFOLFOX 6 + Bevacizumab ( oxaliplatino 85 mg/m2 i.v . día 1 , leucovorin 200 mg/m2 i...",5fluorouracilo,1,5fluorouracilo
2639,5-FU,"i.v . día 1 , leucovorin 200 mg/m2 i.v . día 1 , 5-FU 400 mg/m2 en bolo i.v . día 1 , < start > ...",5fluorouracilo,1,5fluorouracilo
1336,67Ga,trocánter izquierdo . La radiología de pelvis mostró una única lesión lítica en la cabeza femora...,gallium67,303,gallium67
4944,99-MDP,"resección hepática parcial , al mes de su diagnostico . Previamente se había realizado un estudi...",99 metilendifosfonato,2,99 metilendifosfonato


El label debe de ser 0 o 1, es decir, si la LF que le corresponde es la que se le asigna. Por ello creamos una etiqueta binaria.

In [94]:
def get_label(row):
    if row['long_form_x'] == row['long_form_y']:
        return 1
    else:
        return 0

In [95]:
data_merged['label'] = data_merged.apply(get_label, axis = 1)

In [96]:
data_merged.head()

,short_form,context,long_form_x,long_form_y,label
0,mm,"243 UI/L , amilasas 427,8 UI/L , lipasa 1298UI/L . Ecografía abdominal : vía biliar poco distend...",milimetro,milimetro,1
1,mg,"que mostraron : Hto 37,8 % Hb 13,4 g/dl , plaquetas 338.000/ml , leucocitos 7.700/ml ( N 46 % ) ...",miligramo,miligramo,1
2,mg,"que mostraron : Hto 37,8 % Hb 13,4 g/dl , plaquetas 338.000/ml , leucocitos 7.700/ml ( N 46 % ) ...",miligramo,miligramar,0
3,mg,"glucosa 79 mg/dl , creatinina 0,55 mg/dl , proteínas 77 g/l , albúmina 41 gl/l , bilirrubina tot...",miligramo,miligramo,1
4,mg,"glucosa 79 mg/dl , creatinina 0,55 mg/dl , proteínas 77 g/l , albúmina 41 gl/l , bilirrubina tot...",miligramo,miligramar,0


In [97]:
data_merged = data_merged[['short_form', 'context', 'long_form_y', 'label']]

In [98]:
data_merged = data_merged.rename(columns = {'long_form_y':'long_form'})

In [99]:
data_merged.reset_index(drop = True, inplace = True)

In [100]:
data_merged.head()

,short_form,context,long_form,label
0,mm,"243 UI/L , amilasas 427,8 UI/L , lipasa 1298UI/L . Ecografía abdominal : vía biliar poco distend...",milimetro,1
1,mg,"que mostraron : Hto 37,8 % Hb 13,4 g/dl , plaquetas 338.000/ml , leucocitos 7.700/ml ( N 46 % ) ...",miligramo,1
2,mg,"que mostraron : Hto 37,8 % Hb 13,4 g/dl , plaquetas 338.000/ml , leucocitos 7.700/ml ( N 46 % ) ...",miligramar,0
3,mg,"glucosa 79 mg/dl , creatinina 0,55 mg/dl , proteínas 77 g/l , albúmina 41 gl/l , bilirrubina tot...",miligramo,1
4,mg,"glucosa 79 mg/dl , creatinina 0,55 mg/dl , proteínas 77 g/l , albúmina 41 gl/l , bilirrubina tot...",miligramar,0


In [101]:
data_merged.to_csv('test_data_beto.csv')